In [12]:
import threading
import time

lock = threading.Lock()   # Create a Lock to prevent mixed output from multiple threads

def print_num():
    for i in range(5):
        with lock:        # Acquire lock so only one thread prints at a time
            print(f"Numbers are : {i}")
        time.sleep(1)
def letters():
    for letter in "Shlok":
        with lock:        # Use lock to avoid print collision
            print(f"Letters are : {letter}")
        time.sleep(1)

start_time = time.perf_counter()  

t1 = threading.Thread(target=print_num)  #Creating Target Function
t2 = threading.Thread(target=letters)    #Creating Target Function

t1.start()                # Start execution of thread t1
t2.start()                # Start execution of thread t2

t1.join()                 # Main thread waits until t1 finishes execution
t2.join()                 # Main thread waits until t2 finishes execution

end_time = time.perf_counter()  

print(f"\nFinished in {end_time - start_time:.2f} seconds")

#In multithreading, print() without a Lock = messy output

Numbers are : 0
Letters are : S
Numbers are : 1
Letters are : h
Numbers are : 2
Letters are : l
Numbers are : 3
Letters are : o
Letters are : k
Numbers are : 4

Finished in 5.04 seconds


In [3]:
#-------MultiProcessing--------------------
# Processes that runs in parallel 
#CPU Bound tasks that are heavy on CPU usage (ex-Mathematical computation)
# Parallel Execution - Multiple cores on CPU
import multiprocessing
import time
def square():
    for i in range (5):
        time.sleep(1)
        print(f"Square of {i} is : {i*i}")

def cube():
    for i in range (5):
        time.sleep(1)
        print(f"Cube of {i} is : {i*i*i}")

if __name__=="__main__":
    #create 2 process
    p1 = multiprocessing.Process(target=square)
    p2 = multiprocessing.Process(target=cube)
    t=time.time()
    
    #Start process
    p1.start()
    p2.start()
    
    #Waiting for process to complete
    p1.join()
    p2.join()
    
    finished_time = time.time()-t
    print(finished_time)

#We are not getting any output because Multiprocessing is designed for scripts, not notebooks.
#It works well with .py files in PyCharm

0.13599634170532227
